<a href="https://colab.research.google.com/github/BlairSonnen/Glassdoor_AIBot/blob/blairdev/Glassdoor_Ai_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `newtoken` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate

In [ ]:
# simple_rag_csv_ingestion.py

import subprocess
import sys

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Ensure required packages are installed
required_packages = [
    "sentence_transformers",
    "chromadb",
    "pandas",
    "tqdm"
]

for pkg in required_packages:
    install_if_missing(pkg)

from sentence_transformers import SentenceTransformer
from chromadb import PersistentClient
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# Mount Google Drive if in Colab
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except ImportError:
    pass

# Set ChromaDB path to Google Drive for persistence
CHROMA_PATH = "/content/drive/MyDrive/Glassdoor Chroma Store"
Path(CHROMA_PATH).mkdir(parents=True, exist_ok=True)

# Initialize embedding model and ChromaDB client
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
client = PersistentClient(path=CHROMA_PATH)
collection = client.get_or_create_collection("csv_documents")

# Ingest a single CSV file
def ingest_csv(filepath):
    try:
        df = pd.read_csv(filepath, low_memory=False)
        text = df.to_string(index=False)
        return (text, str(filepath))
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return None

# Process and store CSV files
def process_csv_files(folder_path):
    folder = Path(folder_path)
    csv_files = list(folder.glob("**/*.csv"))

    batch = []
    for filepath in tqdm(csv_files, desc="Ingesting CSV files"):
        result = ingest_csv(filepath)
        if result:
            batch.append(result)

    if batch:
        texts, metadatas = zip(*batch)
        embeddings = embed_model.encode(list(texts))
        collection.add(
            documents=list(texts),
            embeddings=[e.tolist() for e in embeddings],
            metadatas=[{"filename": m} for m in metadatas],
            ids=[str(i) for i in range(len(texts))]  # use index as unique ID
        )
        client.persist()

    print(f"Ingested {len(batch)} CSV files.")

# Entry point for Google Colab
if __name__ == "__main__":
    # Update the path below to your Google Drive CSV folder
    process_csv_files("/content/drive/MyDrive/AI Chatbot Data")


Mounted at /content/drive


InternalError: Database error: error returned from database: (code: 266) disk I/O error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ai_recruiter_with_rag.py

import subprocess
import sys
import importlib.util

# Auto-install and upgrade missing packages
required = {
    "torch": "torch",
    "pandas": "pandas",
    "mammoth": "mammoth",
    "docx": "python-docx",
    "fitz": "PyMuPDF",
    "xlrd": "xlrd",
    "sentence_transformers": "sentence-transformers",
    "transformers": "transformers --upgrade",
    "gradio": "gradio",
    "bitsandbytes": "git+https://github.com/TimDettmers/bitsandbytes.git",
    "accelerate": "accelerate --upgrade",
    "chromadb": "chromadb"
}

def install_missing(pkg_map):
    for imp_name, pip_cmd in pkg_map.items():
        if importlib.util.find_spec(imp_name) is None:
            print(f"📦 Installing {pip_cmd}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install"] + pip_cmd.split())

install_missing(required)

import os
import io
import torch
import pandas as pd
import mammoth
import docx
import fitz
import xlrd
import numpy as np
from io import StringIO
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import gradio as gr
from chromadb import PersistentClient

# Mount Google Drive if in Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

# Connect to ChromaDB in Google Drive
CHROMA_PATH = "/content/drive/MyDrive/Glassdoor Chroma Store"
client = PersistentClient(path=CHROMA_PATH)
collection = client.get_or_create_collection("csv_documents")

# Load model with fallback if bitsandbytes fails
model_path = "mistralai/Mistral-7B-Instruct-v0.2"
try:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.float16
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        quantization_config=bnb_config,
        trust_remote_code=True
    )
    print("✅ Loaded model with 4-bit quantization (bnb)")
except Exception as e:
    print("⚠️ Failed to load with bitsandbytes, falling back to fp16")
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )

# Tokenizer and embedder
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# Document loader

def extract_text_from_file(file):
    if file is None:
        return ""
    name = file.name
    ext = name.lower().split(".")[-1]
    content = ""
    bytes_data = file.read()

    if ext == "txt":
        content = bytes_data.decode("utf-8", errors="ignore")
    elif ext == "pdf":
        with fitz.open(stream=bytes_data, filetype="pdf") as doc:
            content = "\n".join(page.get_text() for page in doc)
    elif ext == "docx":
        doc = docx.Document(io.BytesIO(bytes_data))
        content = "\n".join([p.text for p in doc.paragraphs])
    elif ext == "csv":
        content = pd.read_csv(StringIO(bytes_data.decode("utf-8", errors="ignore"))).to_string()
    elif ext in ["xlsx", "xls"]:
        content = pd.read_excel(io.BytesIO(bytes_data)).to_string()
    elif ext == "doc":
        result = mammoth.extract_raw_text(io.BytesIO(bytes_data))
        content = result.value

    return content[:3000] if content else ""

# QA logic with RAG

def qa_with_llm(file, user_option, question):
    if file:
        context = extract_text_from_file(file)
    else:
        query_embedding = embed_model.encode(question).tolist()
        results = collection.query(query_embeddings=[query_embedding], n_results=3)
        context = "\n\n".join(results['documents'][0]) if results and results['documents'] else ""

    if not question.strip():
        if not context:
            return "⚠️ Please upload a document or enter a question."
        question = f"Please {user_option.lower()} the following content:\n\n{context}"

    prompt = f"""
Persona:
You are a knowledgeable and trustworthy talent acquisition professional with deep insight into company cultures, job roles, employee sentiment, and workplace trends. Your role is to help users explore and evaluate potential employers based on real employee feedback.

Task:
Your task is to analyze job reviews and provide helpful, unbiased summaries of a company's work environment, culture, benefits, challenges, and employee satisfaction based on the Glassdoor Job Reviews 2 dataset. You may be asked to compare companies, highlight pros and cons, or give advice on career moves.

Context:
Users are actively exploring new career opportunities and are using this chatbot to research companies they might want to work for. They are seeking honest, concise, and insightful perspectives drawn from employee reviews to help them make informed decisions about where to apply or accept offers.

Format:
Your responses should be structured in clear, digestible paragraphs or bullet points, depending on the question. When appropriate, use sections like:

Overview
Pros
Cons
Common Themes
Advice to Management
Use plain, professional language. Avoid jargon and maintain a helpful, neutral tone.
{context[:1000]}
Question: {question}
Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output = model.generate(**inputs, max_new_tokens=512, temperature=0.7, do_sample=True)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    final_answer = answer.split("Answer:")[-1].strip()

    return final_answer

# Gradio UI

def run_gradio():
    gr.Interface(
        fn=qa_with_llm,
        inputs=[
            gr.File(label="Upload Document"),
            gr.Radio([
                "Summarize the content",
                "Check for grammar/spelling/formatting",
                "Analyze the data"
            ], label="Task"),
            gr.Textbox(label="Ask a question", placeholder="What are the top-rated companies for software engineers?")
        ],
        outputs="text",
        title="AI Recruiter Assistant",
        description="Upload a job-related document and/or ask the AI for recommendations or summaries."
    ).launch(share=True, debug=True)

run_gradio()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

⚠️ Failed to load with bitsandbytes, falling back to fp16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://25b39b77c76bfa7635.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
